In [1]:
from CONSTANTS_MAIN import NUMBER_OF_PERIODS, NUM_WORLD_STATES, NUM_TRAIN_ITER
from schedEvaluate import schedulerEvaluate
from RL.schedImprove import schedulerImprove

import pickle

SCHEDULER_FILE = './schedulers/scheduler1.pickle'


In [3]:
# Load scheduler file
pickle_in = open(SCHEDULER_FILE, 'rb')
scheduler = pickle.load(pickle_in)
pickle_in.close()

print(scheduler)


[[ 0.17224405  0.          0.82775595  0.        ]
 [ 0.14602418  0.50230566  0.026908    0.32476217]
 [ 0.14510703  0.02757201  0.74055114  0.08676983]
 [ 0.          0.17201816  0.          0.82798184]]


In [4]:

from CONSTANTS_MAIN import NUMBER_OF_PERIODS, SUCCESS_THRESHOLD
from RL.CONSTANTS_RL import SCHEDULER_TRAIN_ITER, NUM_WORLD_STATES

from numpy import array, zeros, random, divide, zeros
import multiprocessing as mp
from functools import partial

from simulation import simulate_driver


In [5]:
# Per thread call to orun a single simulation iteration
def callSimulation(i):
    global tempSched 

    # generate moves
    moves = [random.randint(NUM_WORLD_STATES)]
    for i in range(NUMBER_OF_PERIODS - 1):
        moves.append(random.choice([0, 1, 2, 3], p=tempSched[moves[i]]))

    # Run simulation
    avg_return, risk, sharpe = simulate_driver(moves)
    
    # Limit to -3 <= sharpe <= 3
    sharpe = max(-3, min(3, sharpe))

    # whether or update R+ or R- 
    index = 1 if (sharpe < SUCCESS_THRESHOLD) else 0

    # Reinforcement feedback
    R = zeros((2, 4, 4))
    for i in range(len(moves) - 1):
        state, action = moves[i:i+2]

        R[index][state][action] += 1

    return R

In [7]:
jobs = range(4)
CS = partial(callSimulation, scheduler)
pool = mp.Pool()

In [ ]:
 R_RES = pool.map(callSimulation, jobs)